In [2]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import os
from apache_beam import window
from apache_beam.transforms.trigger import AfterWatermark, AfterProcessingTime, AccumulationMode, AfterCount, Repeatedly
from datetime import datetime

# Weapon Score

In [ ]:
def custom_timestamp(elements):
    unix_timestamp = elements[16].rstrip().lstrip()
    return beam.window.TimestampedValue(elements, int(unix_timestamp))

In [4]:
def encode_byte_string(element):
    element = str(element)
    return element.encode('utf-8')

In [17]:
def battle_condition(battle_time, total_points):
    if battle_time >= 10 and battle_time <= 20:
        total_points += 4  
    elif battle_time >=21 and battle_time <= 30:
        total_points += 3
    elif battle_time >=31 and battle_time <=40:
        total_points += 2
    elif battle_time > 40:
        total_points += 1
    return total_points

In [23]:
def weapon_ranking_diff(player_weapon_ranking, opp_weapon_ranking, total_points):
    diff = player_weapon_ranking - opp_weapon_ranking
    if diff >= 6:
        total_points += 3 
    elif diff >= 3:
        total_points += 2 
    else: 
        total_points += 1 
    return total_points

In [24]:
def calculate_battle_points(element_list):     
    total_points = 0
    game_id = element_list[0]
    player_id = element_list[1]                                        
    weapon = element_list[5]                                           

    my_weapon_ranking = element_list[6].rstrip().lstrip()             
    my_weapon_ranking = int(my_weapon_ranking)
    opp_weapon_ranking = element_list[13].rstrip().lstrip()           
    opp_weapon_ranking = int(opp_weapon_ranking)
    total_points = weapon_ranking_diff(my_weapon_ranking, opp_weapon_ranking, total_points)           
    
    battle_time = element_list[15]
    battle_time = int(battle_time.rstrip().lstrip())                  
    total_points = battle_condition(battle_time, total_points) 

        
    my_map_location = element_list[7].rstrip().lstrip()               
    opp_map_location = element_list[14].rstrip().lstrip()  
    if my_map_location != opp_map_location:                         
        total_points += 3         

    return game_id + ':' + player_id + ':' + weapon, total_points   

In [25]:
class PointFn(beam.CombineFn):
    def create_accumulator(self):
        return (0.0, 0)

    def add_input(self, sum_count, input):                         
        (sum, count) = sum_count                                        
        return sum + input, count + 1                                

    def merge_accumulators(self, accumulators):
        sums, counts = zip(*accumulators)                            
        return sum(sums), sum(counts)                               

    def extract_output(self, sum_count):
        (sum, count) = sum_count                                   
        return sum / count if count else float('NaN')                             


In [26]:
def format_result(key_value_pair):
    name, points = key_value_pair
    name_list = name.split(':')
    game_id = name_list[0]
    player_id = name_list[1]
    weapon = ' '.join(name_list[2:])
    return  game_id + ',' + player_id + ', ' + weapon + ', ' + str(points) + ' average battle points '

In [27]:
p = beam.Pipeline()
pubsub_data = (
        p 
        | 'Read from pub sub' >> beam.io.ReadFromText("mobile_game.txt") 
        | 'Parse data' >> beam.Map(lambda element: element.split(','))
        | 'Calculate battle points' >> beam.Map(calculate_battle_points)       
        | 'Window for player' >> beam.WindowInto(window.Sessions(30))
        | 'Group by key' >> beam.CombinePerKey(PointFn())                    
        | 'Format results' >> beam.Map(format_result)    
        | 'Encode data to byte string' >> beam.Map(encode_byte_string)
        | 'Write player score to pub sub' >> beam.io.WriteToText("output/pubsub")
)

result = p.run()
result.wait_until_finish()

'DONE'

In [28]:
!{"head -n 5 output/pubsub-00000-of-00001"}

GM_1,PL_1, BN60, 9.0 average battle points 
GM_1,PL_3, Rifle, 5.5 average battle points 
GM_1,PL_5, Rifle, 6.0 average battle points 
GM_1,PL_10, Bomb, 5.4 average battle points 
GM_1,PL_10, Rifle, 5.384615384615385 average battle points 


-------------------

# Player Score

In [29]:
def custom_timestamp(elements):
    unix_timestamp = elements[16].rstrip().lstrip()
    return beam.window.TimestampedValue(elements, int(unix_timestamp))

In [30]:
def encode_byte_string(element):
    element = str(element)
    return element.encode('utf-8')

In [31]:
def player_pair(element_list):
    return element_list[1],1

In [32]:
def score_pair(element_list):
    return element_list[3],1

In [47]:
p = beam.Pipeline()

In [48]:
pubsub_data = (
    p 
    | 'Read from pub sub' >> beam.io.ReadFromText("mobile_game.txt")
    | 'Parse data' >> beam.Map(lambda element: element.split(','))
    | 'Apply custom timestamp' >> beam.Map(custom_timestamp)
  )

In [49]:
player_score = (
    pubsub_data 
    | 'Form k,v pair of (player_id, 1)' >> beam.Map( player_pair )
    | 'Window for player' >> beam.WindowInto(window.GlobalWindows(), trigger=Repeatedly(AfterCount(10)), accumulation_mode=AccumulationMode.ACCUMULATING) 
    | 'Group players and their score' >> beam.CombinePerKey(sum)
    | 'Encode player info to byte string' >> beam.Map(encode_byte_string)
    | 'Write player score to pub sub' >> beam.io.WriteToText("output/player")
  )

In [54]:
!{"head -n 5 output/player-00000-of-00001"}

('PL_1', 33)
('PL_3', 41)
('PL_5', 14)
('PL_10', 82)
('PL_9', 89)


In [51]:
team_score = (
                pubsub_data 
                | 'Form k,v pair of (team_score, 1)' >> beam.Map( score_pair )
                | 'Window for team' >> beam.WindowInto(window.GlobalWindows(), trigger=Repeatedly(AfterCount(10)), accumulation_mode=AccumulationMode.ACCUMULATING) 
                | 'Group teams and their score' >> beam.CombinePerKey(sum)
                | 'Encode teams info to byte string' >> beam.Map(encode_byte_string)
                | 'Write team score to pub sub' >> beam.io.WriteToText("output/team")
              )

In [52]:
result = p.run()
result.wait_until_finish()

'DONE'

In [53]:
!{"head -n 5 output/team-00000-of-00001"}

('TM_01', 66)
('TM_03', 72)
('TM_05', 82)
('TM_04', 90)
('TM_02', 81)
